In [ ]:
import cv2
import numpy as np
import urllib.request
from ultralytics import YOLO

# Cargar el modelo YOLOv8 con segmentación
model = YOLO('modelo_yolov8s_insectos.pt')  # Debe estar en el mismo directorio

# Dirección IP de tu ESP32-CAM
url = 'Escribe tu dirección IP'
winName = 'ESP32-CAM - YOLOv8 Segmentación'
cv2.namedWindow(winName, cv2.WINDOW_AUTOSIZE)

# Clases del modelo
class_names = ['grillo', 'oruga']

# Umbrales de confianza personalizados por clase (índice de clase)
confidence_thresholds = {
    0: 0.30,  # grillo
    1: 0.75   # oruga
}

while True:
    try:
        # Leer la imagen desde la ESP32-CAM
        imgResp = urllib.request.urlopen(url)
        imgNp = np.array(bytearray(imgResp.read()), dtype=np.uint8)
        frame = cv2.imdecode(imgNp, -1)

        # (Opcional) Rotar si es necesario
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        # Hacer la predicción (con confianza baja)
        results = model.predict(source=frame, conf=0.05, imgsz=640, device='cpu')[0]

        # Dibujar segmentaciones y cajas
        annotated_frame = frame.copy()

        if results.masks is not None:
            for mask, box, cls_id, conf in zip(results.masks.data, results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
                cls_id = int(cls_id)
                conf = float(conf)

                # Filtrar según el umbral por clase
                if cls_id in confidence_thresholds and conf < confidence_thresholds[cls_id]:
                    continue  # No cumple con el mínimo para esta clase

                color = (0, 255, 0)
                label = f"{class_names[cls_id]} ({conf*100:.1f}%)"

                # Dibujar la máscara
                mask = mask.cpu().numpy()
                mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
                mask = (mask > 0.5).astype(np.uint8) * 255

                # Aplicar máscara con transparencia
                colored_mask = np.zeros_like(frame)
                colored_mask[:, :, 1] = mask  # canal verde
                annotated_frame = cv2.addWeighted(annotated_frame, 1.0, colored_mask, 0.5, 0)

                # Dibujar caja
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Mostrar imagen final
        cv2.imshow(winName, annotated_frame)

    except Exception as e:
        print("Error:", e)

    if cv2.waitKey(5) & 0xFF == 27:  # ESC para salir
        break

cv2.destroyAllWindows()



0: 640x480 2 orugas, 372.9ms
Speed: 2.4ms preprocess, 372.9ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 3 orugas, 417.8ms
Speed: 3.2ms preprocess, 417.8ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 grillos, 5 orugas, 376.0ms
Speed: 6.7ms preprocess, 376.0ms inference, 6.4ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 orugas, 423.5ms
Speed: 10.0ms preprocess, 423.5ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 3 orugas, 326.8ms
Speed: 5.3ms preprocess, 326.8ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 orugas, 376.1ms
Speed: 6.9ms preprocess, 376.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 5 orugas, 303.3ms
Speed: 2.4ms preprocess, 303.3ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 grillo, 2 orugas, 361.6ms
Speed: 5.1ms preprocess, 361.6ms inference, 4.6ms po

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Cargar el modelo YOLOv8 con segmentación
model = YOLO('modelo_yolov8s_insectos.pt')  

# Usar la webcam (0 = cámara por defecto)
cap = cv2.VideoCapture(0)
winName = 'Webcam - YOLOv8 Segmentación'
cv2.namedWindow(winName, cv2.WINDOW_AUTOSIZE)

# Clases del modelo
class_names = ['grillo', 'oruga']

# Umbrales de confianza personalizados por clase
confidence_thresholds = {
    0: 0.30,  # grillo
    1: 0.75   # oruga
}

while True:
    ret, frame = cap.read()
    if not ret:
        print("No se pudo leer la cámara.")
        break

    # Hacer la predicción (confianza baja)
    results = model.predict(source=frame, conf=0.05, imgsz=640, device='cpu')[0]

    # Copiar imagen para anotaciones
    annotated_frame = frame.copy()

    if results.masks is not None:
        for mask, box, cls_id, conf in zip(results.masks.data, results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
            cls_id = int(cls_id)
            conf = float(conf)

            # Filtrar por umbral por clase
            if cls_id in confidence_thresholds and conf < confidence_thresholds[cls_id]:
                continue

            color = (0, 255, 0)
            label = f"{class_names[cls_id]} ({conf*100:.1f}%)"

            # Dibujar la máscara
            mask = mask.cpu().numpy()
            mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
            mask = (mask > 0.5).astype(np.uint8) * 255

            colored_mask = np.zeros_like(frame)
            colored_mask[:, :, 1] = mask  # canal verde
            annotated_frame = cv2.addWeighted(annotated_frame, 1.0, colored_mask, 0.5, 0)

            # Dibujar la caja
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated_frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Mostrar imagen
    cv2.imshow(winName, annotated_frame)

    if cv2.waitKey(5) & 0xFF == 27:  # ESC para salir
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 4 orugas, 345.1ms
Speed: 1.8ms preprocess, 345.1ms inference, 12.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 orugas, 353.2ms
Speed: 2.9ms preprocess, 353.2ms inference, 4.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 orugas, 369.8ms
Speed: 6.2ms preprocess, 369.8ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 orugas, 314.3ms
Speed: 1.4ms preprocess, 314.3ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 orugas, 305.7ms
Speed: 2.4ms preprocess, 305.7ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 orugas, 318.2ms
Speed: 1.4ms preprocess, 318.2ms inference, 5.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 orugas, 364.8ms
Speed: 2.4ms preprocess, 364.8ms inference, 6.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 orugas, 363.4ms
Speed: 1.8ms preprocess, 363.4ms inference, 4.6ms postprocess per image a

##### from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Cargar modelo YOLOv8 entrenado
model = YOLO("best.pt")  # Asegúrate de que esté en el mismo directorio o da la ruta completa

# Cargar imagen de prueba
img_path = "Bichos/Images/0AQOFNQ7KU1V.jpg"  # Reemplaza con tu imagen
img = cv2.imread(img_path)

# Asegúrate de que cargó la imagen
if img is None:
    raise FileNotFoundError(f"No se encontró la imagen: {img_path}")

# Ejecutar predicción con confianza mínima de 0.25 (ajustable)
results = model.predict(source=img_path, conf=0.1, save=False, show=True)  # show=True para ver ventana OpenCV

# Mostrar con Matplotlib si prefieres
for result in results:
    annotated = result.plot()  # Imagen con cajas dibujadas
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Resultado de la detección")
    plt.show()

# Mostrar clases detectadas y sus confianzas
for result in results:
    print("Clases detectadas:", result.names)
    print("Resultados:")
    for box in result.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        print(f"Clase: {result.names[cls_id]} | Confianza: {conf:.2f}")
